# My first graph in tensorflow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk

/home/ec2-user/anaconda3/envs/tf+sklearn_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
import tensorflow as tf

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [4]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    result = theta.eval()

print(result)

[[-3.63715744e+01]
 [ 4.37795252e-01]
 [ 9.53791849e-03]
 [-1.08239025e-01]
 [ 6.47155583e-01]
 [-3.69874488e-06]
 [-3.80105944e-03]
 [-4.15575713e-01]
 [-4.28025067e-01]]


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

In [7]:
import time

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

graphLR = tf.Graph()

with graphLR.as_default():
    X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name ='theta')
    y_pred = tf.matmul(X, theta, name="predictions")
    error = y_pred - y
    mse = tf.losses.mean_squared_error(y,y_pred)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
            init.run()
            t0 = time.time()
            for epoch in range(n_epochs):
                if epoch % 100 == 0:
                    print("Epoch ", epoch, "MSE =", mse.eval(), "Elapsed ", time.time()-t0)
                sess.run(training_op)
            theta_fitted = theta.eval()

print(theta_fitted)
                
    
        

Epoch  0 MSE = 10.448111 Elapsed  0.016701221466064453
Epoch  100 MSE = 5.0486903 Elapsed  0.08058905601501465
Epoch  200 MSE = 4.934358 Elapsed  0.10927104949951172
Epoch  300 MSE = 4.900564 Elapsed  0.13756179809570312
Epoch  400 MSE = 4.876825 Elapsed  0.16614055633544922
Epoch  500 MSE = 4.859188 Elapsed  0.19489812850952148
Epoch  600 MSE = 4.8459883 Elapsed  0.22593164443969727
Epoch  700 MSE = 4.836073 Elapsed  0.2546427249908447
Epoch  800 MSE = 4.8285856 Elapsed  0.2833847999572754
Epoch  900 MSE = 4.822907 Elapsed  0.31200361251831055
[[-0.26465988]
 [ 0.88602585]
 [ 0.16151375]
 [-0.31189695]
 [ 0.31756485]
 [ 0.01032065]
 [-0.04414622]
 [-0.5351892 ]
 [-0.50949734]]


In [8]:
show_graph(graphLR)

In [21]:
tf.reset_default_graph()
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable('threshold')
        w_shape = (int((X.get_shape()[1])),1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name = 'bias')
        z = tf.add(tf.matmul(X,w), b)
        return tf.maximum(z, threshold, name ='max')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

with tf.variable_scope("relu"):
    tf.get_variable("threshold", shape=()
                    ,initializer=tf.constant_initializer(0.))
    


relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

    

In [22]:
show_graph(tf.get_default_graph())